In [1]:
import pandas as pd
import zipfile
import kaggle

In [2]:
# downloading zip dataset
!kaggle datasets download -d hmavrodiev/london-bike-sharing-dataset

london-bike-sharing-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
# extracting dataset from zip
zip_file_name = 'london-bike-sharing-dataset.zip'
with zipfile.ZipFile(zip_file_name, 'r') as file:
    file.extractall()

In [4]:
# loading dataset into dataframe
bikes = pd.read_csv('london_merged.csv')

In [5]:
# exploring data
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     17414 non-null  object 
 1   cnt           17414 non-null  int64  
 2   t1            17414 non-null  float64
 3   t2            17414 non-null  float64
 4   hum           17414 non-null  float64
 5   wind_speed    17414 non-null  float64
 6   weather_code  17414 non-null  float64
 7   is_holiday    17414 non-null  float64
 8   is_weekend    17414 non-null  float64
 9   season        17414 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.3+ MB


##### Context on metadata features:</br>

0. `timestamp`: timestamp field for grouping the data
1. `cnt`: the count of new bike shares
2. `t1`: real temparature in C
3. `t2`: temperature in C "feels like"
4. `hum`: humidity in percentage
5. `wind_speed`: wind speed in km/h
6. `weather_code`: category of the weather
7. `is_holiday`: boolean field - 1 holiday / 0 non holiday
8. `is_weekend`: boolean field - 1 weekend / 0 weekday
9. `season`: category field - 0-spring, 1-summer, 2-fall, and 3-winter

- `weather_code` descriptions: 1-clear, 2-few clouds, 3-broken clouds, 4-cloudy, 7-light rain, 10-rain with thunderstorms, 26-snowfall, 94-freezing fog

In [6]:
bikes.shape

(17414, 10)

In [7]:
# counting unique values in weather_code col
bikes.weather_code.value_counts()

1.0     6150
2.0     4034
3.0     3551
7.0     2141
4.0     1464
26.0      60
10.0      14
Name: weather_code, dtype: int64

In [8]:
# counting unique values in season col
bikes.season.value_counts()

0.0    4394
1.0    4387
3.0    4330
2.0    4303
Name: season, dtype: int64

In [9]:
# defining column names we want use
new_col_names = {
    'timestamp':'time',
    'cnt':'count',
    't1':'temp_real_C',
    't2':'temp_feels_like_C',
    'hum':'humidity_percent',
    'wind_speed':'wind_speed_kph',
    'weather_code':'weather',
    'is_holiday':'is_holiday',
    'is_weekend':'is_weekend',
    'season':'season'
}
# renaming columns in dataframe
bikes.rename(new_col_names, axis=1, inplace=True)

In [10]:
# changing humidity values into % aka between 0 and 1
bikes.humidity_percent = bikes.humidity_percent /100

In [11]:
# season dictionary to map numeric values to string values
season_dict = {
    '0.0':'spring',
    '1.0':'summer',
    '2.0':'fall',
    '3.0':'winter'
}

# weather dictionary to map numeric values to string values
weather_dict = {
    '1.0':'Clear',
    '2.0':'Scattered clouds',
    '3.0':'Broken clouds',
    '4.0':'Cloudy',
    '7.0':'Rain',
    '10.0':'Rain with thunderstorms',
    '26.0':'Snowfall'
}

# changing season and weather datatype to string
bikes.season = bikes.season.astype('str')
bikes.weather = bikes.weather.astype('str')

# mapping numeric values to string values
bikes.season = bikes.season.map(season_dict)
bikes.weather = bikes.weather.map(weather_dict)

In [12]:
# verifying mapping worked
bikes.head()

,time,count,temp_real_C,temp_feels_like_C,humidity_percent,wind_speed_kph,weather,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,0.930,6.0,Broken clouds,0.0,1.0,winter
1,2015-01-04 01:00:00,138,3.0,2.5,0.930,5.0,Clear,0.0,1.0,winter
2,2015-01-04 02:00:00,134,2.5,2.5,0.965,0.0,Clear,0.0,1.0,winter
3,2015-01-04 03:00:00,72,2.0,2.0,1.000,0.0,Clear,0.0,1.0,winter
4,2015-01-04 04:00:00,47,2.0,0.0,0.930,6.5,Clear,0.0,1.0,winter


In [13]:
# writing final dataframe to an excel file for Tableau visualisations.
bikes.to_excel('london_bikes_final.xlsx', sheet_name='Data')